In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import os
import shutil
import pickle as pk

from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from sklearn import metrics

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/Satelite_Numeric_data/New Dataset 5000.csv")
data.head(5)

StationId        Datetime  PM2.5  PM10    NO    NO2    NOx    NH3    CO  \
0     TG002  5/14/2018 0:00  35.25  88.5  2.12  33.10  19.30  14.10  0.96   
1     TG002  5/14/2018 1:00  32.00  73.5  2.17  20.30  12.60  13.80  0.17   
2     TG002  5/14/2018 2:00  20.50  58.0  3.00  17.35  11.65  13.48  1.53   
3     TG002  5/14/2018 3:00  21.00  50.0  2.73  18.28  11.93  13.50  2.96   
4     TG002  5/14/2018 4:00  18.50  43.5  2.20  16.38  10.45  15.13  3.49   

    SO2     O3  Benzene  Toluene  Xylene  AQI AQI_Bucket  
0  1.97  67.25     4.98     7.43    2.70  168   Moderate  
1  2.77  80.75     3.40     4.27    1.57  124   Moderate  
2  2.95  77.22     2.02     2.55    1.18  124   Moderate  
3  2.02  36.35     1.42     1.57    0.80  124   Moderate  
4  2.20  70.90     1.20     1.10    1.03  124   Moderate

In [ ]:
df = data.drop(['StationId', 'Datetime', 'AQI_Bucket'], axis=1)
df

PM2.5    PM10     NO     NO2    NOx    NH3    CO   SO2     O3  Benzene  \
0     35.25   88.50   2.12   33.10  19.30  14.10  0.96  1.97  67.25     4.98   
1     32.00   73.50   2.17   20.30  12.60  13.80  0.17  2.77  80.75     3.40   
2     20.50   58.00   3.00   17.35  11.65  13.48  1.53  2.95  77.22     2.02   
3     21.00   50.00   2.73   18.28  11.93  13.50  2.96  2.02  36.35     1.42   
4     18.50   43.50   2.20   16.38  10.45  15.13  3.49  2.20  70.90     1.20   
...     ...     ...    ...     ...    ...    ...   ...   ...    ...      ...   
5086  40.75   93.75   3.55   91.53  51.58  17.98  0.87  3.45  42.12     3.30   
5087  64.25  151.75   4.97  121.20  68.53  21.15  1.21  3.00  13.75     6.22   
5088  82.25  179.50  17.82  142.17  90.10  20.10  1.48  4.05   2.42     8.55   
5089  78.25  165.00  22.60  144.02  94.95  20.75  1.45  3.97   1.20     9.62   
5090  75.50  160.75  12.47  134.22  81.55  21.70  1.29  2.00   4.97     9.65   

      Toluene  Xylene  AQI  
0        7.43    2.70  168  
1        4.27    1.57  124  
2        2.55    1.18  124  
3        1.57    0.80  124  
4        1.10    1.03  124  
...       ...     ...  ...  
5086     7.35    3.00  106  
5087    16.48    8.20  106  
5088    23.40   12.93  106  
5089    24.93   15.65  107  
5090    22.52   15.62  108  

[5091 rows x 13 columns]

In [ ]:
X = df.loc[:, 'NO':'Xylene'].values
y = df['AQI'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3563, 10)
(3563,)
(1528, 10)
(1528,)


In [ ]:
sc=StandardScaler()

scaler = sc.fit(X_train)

trainX_scaled = scaler.transform(X_train)
testX_scaled = scaler.transform(X_test)

In [ ]:
checkpoint_no = 'ckpt_1_NN'
model_name = 'satellite_ANN_2FC_F64_64_epoch_120'

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# create ANN model
model = Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=10, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))

# The output neuron is a single fully connected node
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
model.fit(trainX_scaled, y_train ,batch_size = 20, epochs = 50, verbose=1)

Epoch 1/50
179/179 [==============================] - 2s 3ms/step - loss: 10242.4629
Epoch 2/50
179/179 [==============================] - 1s 3ms/step - loss: 9913.1162
Epoch 3/50
179/179 [==============================] - 1s 3ms/step - loss: 9626.1250
Epoch 4/50
179/179 [==============================] - 1s 3ms/step - loss: 9397.1768
Epoch 5/50
179/179 [==============================] - 1s 3ms/step - loss: 9190.0811
Epoch 6/50
179/179 [==============================] - 1s 3ms/step - loss: 8994.0439
Epoch 7/50
179/179 [==============================] - 1s 3ms/step - loss: 8806.3506
Epoch 8/50
179/179 [==============================] - 1s 3ms/step - loss: 8625.1543
Epoch 9/50
179/179 [==============================] - 1s 3ms/step - loss: 8448.3154
Epoch 10/50
179/179 [==============================] - 1s 3ms/step - loss: 8275.8467
Epoch 11/50
179/179 [==============================] - 1s 3ms/step - loss: 8107.0171
Epoch 12/50
179/179 [==============================] - 1s 3ms/step - loss

In [ ]:
checkpoint_dir = './'+ checkpoint_no
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
keras_callbacks = [ModelCheckpoint(filepath = checkpoint_dir + '/' + model_name,
                                   monitor='val_loss', save_best_only=True, mode='auto')]

In [ ]:
model.evaluate(testX_scaled, y_test)

48/48 [==============================] - 0s 3ms/step - loss: 3369.8147


3369.814697265625

In [ ]:
# Function to generate Deep ANN model
def make_regression_ann(Optimizer_trial='adam'):
    from keras.models import Sequential
    from keras.layers import Dense

    model = Sequential()
    model.add(Dense(units=5, input_dim=10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

###########################################
from sklearn.model_selection import GridSearchCV
import scikeras
from scikeras.wrappers import KerasRegressor


# Listing all the parameters to try
Parameter_Trials={'batch_size':[10,20,30],
                      'epochs':[10,20],
                 }

# Creating the regression ANN model
RegModel=KerasRegressor(make_regression_ann, verbose=0)

###########################################
from sklearn.metrics import make_scorer

# Defining a custom function to calculate accuracy
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

#########################################
# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=RegModel,
                         param_grid=Parameter_Trials,
                         scoring=custom_Scoring,
                         cv=5)

#########################################
# Measuring how much time it took to find the best params
import time
StartTime=time.time()

# Running Grid Search for different paramenters
grid_search.fit(X,y, verbose=1)

EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')

print('### Printing Best parameters ###')
grid_search.best_params_

Epoch 1/10
408/408 [==============================] - 2s 3ms/step - loss: 4551.9165
Epoch 2/10
408/408 [==============================] - 2s 4ms/step - loss: 513.4188
Epoch 3/10
408/408 [==============================] - 1s 3ms/step - loss: 472.6984
Epoch 4/10
408/408 [==============================] - 1s 3ms/step - loss: 452.3552
Epoch 5/10
408/408 [==============================] - 2s 5ms/step - loss: 435.1450
Epoch 6/10
408/408 [==============================] - 2s 4ms/step - loss: 423.3262
Epoch 7/10
408/408 [==============================] - 1s 3ms/step - loss: 414.4035
Epoch 8/10
408/408 [==============================] - 1s 3ms/step - loss: 406.8692
Epoch 9/10
408/408 [==============================] - 1s 3ms/step - loss: 402.0966
Epoch 10/10
408/408 [==============================] - 1s 3ms/step - loss: 398.9123
###################################################################### Accuracy: 78.39588142290984
Epoch 1/10
408/408 [==============================] - 2s 3ms/step - l

{'batch_size': 30, 'epochs': 20}

In [ ]:
!pip install scikeras

In [ ]:
# Fitting the ANN to the Training set
model.fit(trainX_scaled, y_train ,batch_size = 30, epochs = 20, verbose=0)

# Generating Predictions on testing data
Predictions = model.predict(testX_scaled)  # Flatten the predictions


48/48 [==============================] - 0s 2ms/step


In [ ]:
df_testY = pd.DataFrame(y_test)
df_y_pred = pd.DataFrame(Predictions)

df_testY.reset_index(drop=True, inplace=True)
df_y_pred.reset_index(drop=True, inplace=True)

df_results = pd.concat([df_testY, df_y_pred], axis=1)
df_results.columns = ['Actual', 'Predicted']

df_results

Actual   Predicted
0         59   82.560822
1         65   85.606888
2         45   65.174042
3        142  119.462250
4        126  109.282822
...      ...         ...
1523      37   43.423016
1524     109   85.642441
1525     124  120.289124
1526     142  120.272278
1527      91  113.735214

[1528 rows x 2 columns]

In [ ]:
APE = 100 * (abs(df_results['Actual'] - df_results['Predicted']) / df_results['Actual'])
df_results['APE'] = APE

# Calculate the accuracy of the ANN model
accuracy = 100 - np.mean(APE)
print('The Accuracy of ANN model is:', accuracy)
df_results.head()

The Accuracy of ANN model is: 78.82390921961631


Actual   Predicted        APE
0      59   82.560822  39.933596
1      65   85.606888  31.702904
2      45   65.174042  44.831204
3     142  119.462250  15.871655
4     126  109.282822  13.267602

In [ ]:
from sklearn.metrics import r2_score
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Predictions))
print('Root Mean Squared Error:', metrics.mean_squared_error(y_test, Predictions, squared=False))
print('Mean Absolute Percentage Error:', metrics.mean_absolute_percentage_error(y_test, Predictions))
print('r2 score :', r2_score(y_test, Predictions))


Mean Absolute Error: 16.038907058576015
Mean Squared Error: 458.8928972087711
Root Mean Squared Error: 21.42178557470808
Mean Absolute Percentage Error: 0.21176090780383683
r2 score : 0.728562994991265
